# 梯度下降法(Gradient-Descent)

## 原理

### 概念

   - 不是一个机器学习的算法
   - 是一种基于搜索的最优化方法
   - 作用: 最小化一个损失函数
   - 梯度上升法: 最大化一个效用函数

- [](![](https://raw.githubusercontent.com/LinCheungS/PicGo_Image_Storage/master/2020/20200422204644.png)
)

### 步骤

- 先找到目标函数,比如
$$ y=x+b $$
- 求解损失函数$J({\theta})$, 比如用MSE,也叫标准差
$$J=\sum_{i=1}^{m}\left(y^{(i)}-\hat{y}^{(i)}\right)^{2}$$ 
- 求损失函数的导数, 求解最小值或最大值
$$J(\theta)=\frac{1}{2 m} \sum_{i=1}^{m}\left(y^{(i)}-\hat{y}^{(i)}\right)^{2}$$
- 利用梯度公式,实现惩罚机制,更新$\theta$
$$\theta = \theta - \alpha {f_{loss}}'(\theta)$$

### 注意事项

* 学习率的问题
* 要数据归一化
* $X_0$ 添加一列为1
* 某一函数有多个极值点: 解决方法-多次运行-随机化初始点
* 将两次theta至变化小于1e-8时，可以认为到达了极值

## 分类

### 批量梯度下降法(BGD)

- 批量梯度下降法是最原始的形式
- 表达式递推到具有m个训练样本
- 然后对损失函数求导
$$\frac{\partial J\left(\Theta_{0}, \Theta_{1}\right)}{\partial \Theta_{j}}=\frac{1}{m} \sum_{i=1}^{m}\left(H_{\Theta}\left(X^{(i)}\right)-Y^{(i)}\right) X_{j}^{(i)}$$  
$$\Theta_{j}=\Theta_{i}-\alpha \frac{1}{m} \sum_{i=1}^{m}\left(H_{\Theta}\left(X^{(i)}\right)-Y^{(i)}\right) X_{j}^{(i)}$$  

![](https://raw.githubusercontent.com/LinCheungS/PicGo_Image_Storage/master/2020/20200422205146.png)

- 优点：
    1. 一次迭代是对所有样本进行计算，此时利用矩阵进行运算，实现了并行。
    2. 由全数据集确定的方向能够更好地代表样本总体，从而更准确地朝向极值所在的方向。
    3. 当目标函数为凸函数时，批量梯度下降一定能够得到全局最优解
- 缺点
    1. 有时我们会遇到样本数目 m 很大的训练集合，如果有几十上百万，甚至上亿的训练样本。
    2. 这意味着我们每执行一次批梯度下降算法，都要对m个样本进行求和。
    3. 这样会导致，训练过程很慢，花费很长的时间。

### 随机梯度下降法(SGD)

- 随机梯度下降是每次迭代使用一个样本来对参数进行更新
- 使得训练速度加快。
$$\frac{\Delta J^{(i)}\left(\theta_{0}, \theta_{1}\right)}{\theta_{j}}=\left(h_{\theta}\left(x^{(i)}\right)-y^{(i)}\right) x_{j}^{(i)}$$  
$$\theta_{j}:=\theta_{j}-\alpha\left(h_{\theta}\left(x^{(i)}\right)-y^{(i)}\right) x_{j}^{(i)}$$

![](https://raw.githubusercontent.com/LinCheungS/PicGo_Image_Storage/master/2020/20200422205626.png)

- $\eta$ 称为学习率(learning rate)  
- $\eta$ 的取值影响获得最优解的速度  
- $\eta$ 取值不合适，甚至得不到最优解  
- $\eta$ 是梯度下降法的一个超参数  

- 优点：
    1. 由于不是在全部训练数据上的损失函数，而是在每轮迭代中，随机优化某一条训练数据上的损失函数，这样每一轮参数的更新速度大大加快。
- 缺点
    1. 准确度下降。由于即使在目标函数为强凸函数的情况下，SGD仍旧无法做到线性收敛。
    2. 可能会收敛到局部最优，由于单个样本并不能代表全体样本的趋势。
    3. 不易于并行实现。

### 小批量梯度下降法(MBGD)

- 小批量梯度下降算法是介于批量梯度下降算法和随机梯度下降算法之间的算法
- batchsize=100指的是每次取100个样本训练。所以需要迭代300000/100=3000次。

batchsize=3000
$$\theta_{j}=\theta_{j}-\alpha * \frac{1}{3000} \sum_{i}^{3000}\left(h_{\theta}\left(x^{i}\right)-y^{i}\right) * x_{j}^{i}$$

- 优点：
    1. 通过矩阵运算，每次在一个batch上优化神经网络参数并不会比单个数据慢太多。
    2. 每次使用一个batch可以大大减小收敛所需要的迭代次数，同时可以使收敛到的结果更加接近梯度下降的效果。(比如上例中的30W，设置batch_size=100时，需要迭代3000次，远小于随机梯度下降的30W次)
    3. 可实现并行化。
- 缺点：
    1. batch_size的不当选择可能会带来一些问题。

## 底层实现

In [1]:
import numpy as np
from sklearn.metrics import r2_score

class LinearRegression:

    def __init__(self):
        """初始化Linear Regression模型"""
        self.coef_ = None
        self.intercept_ = None
        self._theta = None
    
    #批量梯度下降
    def fit_gd(self, X_train, y_train, eta=0.001, n_iters=1e4):
        """根据训练数据集X_train, y_train, 使用梯度下降法训练Linear Regression模型"""
        assert X_train.shape[0] == y_train.shape[0], \
            "the size of X_train must be equal to the size of y_train"
        
        #求损失函数
        def J(theta, X_b, y):
            try:
                return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
            except:
                return float('inf')
        
        #求导损失函数
        def dJ(theta, X_b, y):
            return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(X_b)
        
        #运行梯度下降
        def gradient_descent(X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-8):
            theta = initial_theta
            cur_iter = 0
            while cur_iter < n_iters:
                gradient = dJ(theta, X_b, y)
                last_theta = theta
                theta = theta - eta * gradient
                if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
                    break
                cur_iter += 1
            return theta

        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        initial_theta = np.zeros(X_b.shape[1])
        self._theta = gradient_descent(X_b, y_train, initial_theta, eta, n_iters)
        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]

        return self
    
    #随机梯度下降
    def fit_sgd(self, X_train, y_train, n_iters=5, t0=5, t1=50):
        """根据训练数据集X_train, y_train, 使用梯度下降法训练Linear Regression模型"""
        assert X_train.shape[0] == y_train.shape[0], \
            "the size of X_train must be equal to the size of y_train"
        assert n_iters >= 1

        def dJ_sgd(theta, X_b_i, y_i):
            return X_b_i * (X_b_i.dot(theta) - y_i) * 2.

        def sgd(X_b, y, initial_theta, n_iters, t0=5, t1=50):

            def learning_rate(t):
                return t0 / (t + t1)
            theta = initial_theta
            m = len(X_b)

            for cur_iter in range(n_iters):
                
                #随机选取一个数据
                indexes = np.random.permutation(m)
                X_b_new = X_b[indexes]
                y_new = y[indexes]
                for i in range(m):
                    gradient = dJ_sgd(theta, X_b_new[i], y_new[i])
                    theta = theta - learning_rate(cur_iter * m + i) * gradient
            return theta

        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        initial_theta = np.random.randn(X_b.shape[1])
        self._theta = sgd(X_b, y_train, initial_theta, n_iters, t0, t1)

        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]

        return self
    
    #预测
    def predict(self, X_predict):
        """给定待预测数据集X_predict，返回表示X_predict的结果向量"""
        assert self.intercept_ is not None and self.coef_ is not None, \
            "must fit before predict!"
        assert X_predict.shape[1] == len(self.coef_), \
            "the feature number of X_predict must be equal to X_train"

        X_b = np.hstack([np.ones((len(X_predict), 1)), X_predict])
        return X_b.dot(self._theta)
    
    #准确率
    def score(self, X_test, y_test):
        """根据测试数据集 X_test 和 y_test 确定当前模型的准确度"""
        y_predict = self.predict(X_test)
        return r2_score(y_test, y_predict)

    def __repr__(self):
        return "LinearRegression()"

In [2]:
from sklearn import datasets

boston = datasets.load_boston()
x = boston.data
y= boston.target

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
standerscaler = StandardScaler()
x = standerscaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

gradient_descent = LinearRegression()
gradient_descent.fit_gd(x_train,y_train)

LinearRegression()

In [4]:
gradient_descent.score(x_test,y_test)

0.763488198838198

## sklearn实现

In [5]:
from sklearn import datasets
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

boston = datasets.load_boston()
x = boston.data
y= boston.target
x = x[y <50.0]
y = y[y <50.0]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

standerscaler = StandardScaler()
standerscaler.fit(x_train)
x_train_scaler = standerscaler.transform(x_train)
x_test_scaler = standerscaler.transform(x_test)

In [6]:
lin_reg = SGDRegressor(tol=1e-4)
lin_reg.fit(x_train_scaler,y_train)
lin_reg.score(x_test_scaler,y_test)

0.716269351692778